# Curso Análisis de Redes

## Cátedra de Teoría de las Telecomunicaciones - Universidad ORT Uruguay 


# Trabajo Obligatorio 2, curso 2020


------

### Datos de entrega:

* **Fecha de entrega:** 14/12/2020
* **Grupos:** Hasta 3 integrantes
* **Modalidad de entrega:** cuaderno en PDF en gestion + repositorio Github

-----

In [1]:
#using Pkg;Pkg.instantiate(); Pkg.update()

## Ejercicio 1

El objetivo de este ejercicio es verificar el modelo de Bianchi de WiFi en condiciones idealizadas (simuladas) y sobre una captura real de WiFi 802.11b realizada en el laboratorio.

### 1a. parte

El siguiente código permite simular en Julia el proceso de Backoff que vimos en WiFi. Se define un objeto (`mutable struct`) que almacena el estado de los contadores internos, y lleva la cuenta de la cantidad de intentos de acceso, éxitos y colisiones obtenidas.

Para construir una instancia, basta con realizar `station = BackoffProcess(CWmin,stages)` siendo `CWmin` la ventana mínima de backoff y `stages` el no. de duplicaciones. Por defecto todos los contadores se inicializan en $0$, la ventana en el mínimo y el contador se sortea uniforme.

In [3]:
using Distributions

mutable struct BackoffProcess
    
    CWmin::Integer ##minimal backoff window
    stages::Integer ##maximum number of duplications
    w::Integer ##current backoff counter
    s::Integer ##current stage
    attempts::Integer
    successes::Integer
    collisions::Integer
    
    BackoffProcess(CWmin::Integer,stages::Integer)= new(CWmin,stages,rand(DiscreteUniform(0,CWmin-1)),0,0,0,0)
end 

Los siguientes métodos permiten actualizar un objeto de tipo `BackoffProcess` en función de si lo que ocurre es un slot vacío, un slot de transmisión exitosa o una colisión.

In [4]:
function update_on_idle!(st::BackoffProcess)
    
    @assert st.w>0 "No deberia haber entrado aca ya que esta estación debía acceder"
    
    st.w=st.w-1
end

function update_on_success!(st::BackoffProcess)
    
    if st.w>0
        st.w=st.w-1
    else
        st.s=0
        st.w=rand(DiscreteUniform(0,st.CWmin-1))
        st.attempts=st.attempts+1
        st.successes=st.successes+1
    end
end

function update_on_collision!(st::BackoffProcess)
    
    if st.w>0
        st.w=st.w-1
    else
        st.s=min(st.s+1,st.stages)
        st.w=rand(DiscreteUniform(0,st.CWmin*2^st.s-1))
        st.attempts=st.attempts+1
        st.collisions=st.collisions+1
    end
end

update_on_collision! (generic function with 1 method)

Se pide:

1. Realice un código basado en los vistos en clase para simular una celda de $N$ estaciones intentando acceder utilizando el proceso de Backoff de WiFi con los parámetros habituales de 802.11b ($CW_{min}=32, CW_{max}=1024$) y los tiempos de slot, transmisión y colisión ya vistos en clase.


2. Simule lo anterior para el caso de $N=5$ estaciones transmitiendo a $1Mbps$ durante 60 segundos. Grafique la evolución temporal del sistema (por ejemplo paquetes transmitidos).


3. Estime de la simulación:
   * El throughput total de la celda (total de paquetes enviados exitosamente sobre tiempo transcurrido).
   * La probabilidad de acceso de cada estación.
   * La probabilidad condicional de colisión de cada estación.
   
   
4. Compare los resultados anteriores con los que surgen del modelo de Bianchi visto en clase.

### 2a. parte

En el archivo `datos/captura_wifi.csv` se dispone del resumen de una captura Wireshark de $N=5$ estaciones en un AP conectadas a $1Mbps$, intentando transmitir en condiciones de saturación. La captura se realizó del lado del Access Point. La misma puede leerse usando el comando `CSV.read("datos/captura_wifi.csv")` del paquete `CSV`. A su vez puede ser útil el paquete `DataFrames` para trabajar con el resultado. 

1. Filtre los datos correspondientes a las transmisiones de cada estación.
2. Grafique la cantidad de paquetes transmitidos en función del tiempo y compare entre estaciones.
3. Compare el throughput total obtenido con el resultante del modelo anterior y comente si el reparto fue equitativo.

## Ejercicio 2

A completar con ejercicio de TCP (se publicará en la semana del 09/11)